In [ ]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.9 MB/s eta 0:00:00


In [23]:
from faker import Faker
import csv
import random
from datetime import datetime, timedelta

fake = Faker('en_GB')  # Use UK locale for generating UK-specific data

# Generate Customer Data
def generate_customer_data(file_path, num_records):
    pass_types = ["Monthly", "Weekly", "Day"]
    genders = ["Male", "Female"]
    with open(file_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["row_wid", "Name", "Phone_Number", "email_id", "Pass_Type", "Pass_Valid_From", "Pass_Valid_To", "Gender", "Age"])

        for i in range(1, num_records + 1):
            name = fake.name()
            phone = fake.phone_number() if random.random() > 0.1 else None
            email = fake.email() if random.random() > 0.05 else None
            pass_type = random.choice(pass_types)
            pass_valid_from = (datetime.now() - timedelta(days=random.randint(0, 30))).date()

            if pass_type == "Weekly":
                pass_valid_to = pass_valid_from + timedelta(weeks=1)
            elif pass_type == "Monthly":
                pass_valid_to = pass_valid_from + timedelta(days=30)
            else:
                pass_valid_to = pass_valid_from

            gender = random.choice(genders)
            age = random.randint(18, 60)
            writer.writerow([i, name, phone, email, pass_type, pass_valid_from, pass_valid_to, gender, age])


# Generate Driver Data
def generate_driver_csv(file_path, num_records):
    with open(file_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(['row_wid', 'Name', 'PhoneNumber', 'EmailID', 'DOB', 'Experience', 'Age'])
        for i in range(1, num_records + 1):
            name = fake.name()
            phone_number = fake.random_int(1000000000, 9999999999)
            email = fake.email()
            dob = fake.date_of_birth(minimum_age=25, maximum_age=60)
            experience = random.randint(1, 30)
            age = datetime.now().year - dob.year
            writer.writerow([i, name, phone_number, email, dob, experience, age])


# Generate Bus Route Data
def generate_bus_route_data(file_path):
    with open(file_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["row_wid", "Route_num", "start_stop", "destination_stop"])
        row_wid = 1
        for i in range(1, 6):
            start_stop = fake.city()
            destination_stop = fake.city()
            writer.writerow([row_wid, i, start_stop, destination_stop])
            row_wid += 1
            writer.writerow([row_wid, i + 5, destination_stop, start_stop])  # Reverse route
            row_wid += 1


# Generate Time Table Data
def generate_time_table_data(bus_routes, time_table_file):
    with open(time_table_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["row_wid", "Route_num", "Stop_name", "Time"])  # TimeTable columns
        row_wid = 1

        for route in bus_routes:
            route_num = int(route["Route_num"])
            start_stop = route["start_stop"]
            destination_stop = route["destination_stop"]

            if route_num > 5:  # Skip reverse routes during forward generation
                continue

            intermediate_stops = [fake.street_name() for _ in range(random.randint(8, 15))]
            all_stops = [start_stop] + intermediate_stops + [destination_stop]

            current_time = datetime.now()
            for stop in all_stops:
                writer.writerow([row_wid, route_num, stop, current_time.strftime('%H:%M:%S')])
                current_time += timedelta(minutes=random.randint(5, 10))
                row_wid += 1

            reverse_route_num = route_num + 5
            reverse_stops = all_stops[::-1]
            reverse_start_time = current_time + timedelta(minutes=15)

            for stop in reverse_stops:
                writer.writerow([row_wid, reverse_route_num, stop, reverse_start_time.strftime('%H:%M:%S')])
                reverse_start_time += timedelta(minutes=random.randint(5, 10))
                row_wid += 1


# Generate Price Data
def generate_price_data(price_file, time_table_data):
    with open(price_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["row_wid", "Route_num", "From_stop", "To_stop", "Price_adult", "Price_child"])
        row_wid = 1

        for route_num, stops in time_table_data.items():
            for i in range(len(stops)):
                for j in range(i + 1, len(stops)):
                    from_stop = stops[i]
                    to_stop = stops[j]
                    distance = j - i
                    price_adult = round(2 + distance * 1.5, 2)
                    price_child = round(price_adult * 0.5, 2)
                    writer.writerow([row_wid, route_num, from_stop, to_stop, price_adult, price_child])
                    row_wid += 1


# Load Time Table Data
def load_time_table(file_path):
    time_table = {}
    with open(file_path, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            route = int(row["Route_num"])
            if route not in time_table:
                time_table[route] = []
            time_table[route].append(row["Stop_name"])
    return time_table


# Generate Ticket Data
# Generate Ticket Data
def generate_ticket_data(file_path, customers, drivers, bus_routes, prices, timetable, min_rows=1100, max_attempts=5000):
    with open(file_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([
            "row_wid", "Customer_wid", "Driver_wid", "Bus_id", "Route_num", "Departure_stop", "Arrival_stop",
            "Price_adult", "Price_child", "no_of_adults", "no_of_children", "pass", "ticket_date_time", "total_price"
        ])
        row_wid = 1
        composite_key_tracker = set()
        attempts = 0  # To track attempts and prevent infinite loops

        # Pre-build a dictionary for prices for faster lookup
        price_dict = {
            (int(price["Route_num"]), price["From_stop"], price["To_stop"]): (float(price["Price_adult"]), float(price["Price_child"]))
            for price in prices
        }

        # Pre-filter valid routes
        valid_routes = [route for route in bus_routes if int(route["Route_num"]) in timetable]

        # Shuffle drivers for better distribution
        random.shuffle(drivers)

        while row_wid <= min_rows and attempts < max_attempts:
            attempts += 1  # Increment the attempt counter

            # Randomly allow Customer_wid to be NULL
            customer = random.choice(customers + [None])
            customer_wid = customer["row_wid"] if customer else None
            has_pass = customer and customer["Pass_Type"] != ""

            # Select a random valid route
            if not valid_routes:
                break  # Exit if there are no valid routes
            route = random.choice(valid_routes)
            route_num = int(route["Route_num"])
            bus_id = route["row_wid"]  # Extract the bus_id from the BusRoute table

            # Retrieve stops for the selected route
            stops = timetable[route_num]
            if len(stops) < 2:  # Skip routes with fewer than 2 stops
                continue

            # Randomly select two different stops
            try:
                departure_stop, arrival_stop = random.sample(stops, 2)
            except ValueError:
                continue  # Skip if fewer than 2 stops available

            # Retrieve price information
            price_key = (route_num, departure_stop, arrival_stop)
            if price_key not in price_dict:
                continue  # Skip if no price entry is found

            price_adult, price_child = price_dict[price_key]

            # Align ticket time with timetable time
            departure_time_index = stops.index(departure_stop)
            ticket_time = fake.date_time_this_month().replace(hour=0, minute=0, second=0) + timedelta(
                minutes=departure_time_index * 10
            )  # Align time with timetable sequence

            # Generate ticket details
            driver = random.choice(drivers)
            driver_wid = driver["row_wid"]
            no_of_adults = random.randint(1, 3)
            no_of_children = random.randint(0, 2)

            # Check for composite key uniqueness
            composite_key = (customer_wid, ticket_time, route_num)
            if composite_key in composite_key_tracker:
                continue  # Skip if the composite key already exists
            composite_key_tracker.add(composite_key)

            # Calculate total price
            total_price = (no_of_adults * price_adult) + (no_of_children * price_child)

            # If the customer has a pass, one adult ticket is free
            if has_pass and no_of_adults > 0:
                total_price -= price_adult

            total_price = max(round(total_price, 2), 0)  # Ensure total price is non-negative

            # Write the ticket data
            writer.writerow([
                row_wid, customer_wid, driver_wid, bus_id, route_num, departure_stop, arrival_stop,
                price_adult, price_child, no_of_adults, no_of_children, "yes" if has_pass else "no",
                ticket_time.strftime('%Y-%m-%d %H:%M:%S'), total_price
            ])
            row_wid += 1

        if row_wid <= min_rows:
            print(f"Warning: Only {row_wid - 1} tickets generated after {max_attempts} attempts.")

# Load CSV Data
def load_csv(file_path):
    try:
        with open(file_path, mode="r") as file:
            return list(csv.DictReader(file))
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return []
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return []

# File Paths
customer_file = "Customer.csv"
driver_file = "Driver.csv"
bus_route_file = "BusRoute.csv"
time_table_file = "TimeTable.csv"
price_file = "Price.csv"
ticket_file = "Ticket_stage.csv"

# Generate Data
generate_customer_data(customer_file, 50)
generate_driver_csv(driver_file, 50)
generate_bus_route_data(bus_route_file)

bus_routes = load_csv(bus_route_file)
generate_time_table_data(bus_routes, time_table_file)

time_table_data = load_time_table(time_table_file)
generate_price_data(price_file, time_table_data)

customers = load_csv(customer_file)
drivers = load_csv(driver_file)
prices = load_csv(price_file)

generate_ticket_data(ticket_file, customers, drivers, bus_routes, prices, time_table_data, min_rows=1100)

print("Data generation completed successfully!")


Data generation completed successfully!
